In [1]:
import pandas as pd
from pathlib import Path

In [4]:
basedir = Path("/Users/jbirgmei/prog/sspsygene")
print(basedir)

/Users/jbirgmei/prog/sspsygene


In [9]:
mouse_perturb_dir = basedir / "data" / "pheno" / "data" / "mouse-perturb-4tf"
mouse_perturb_deg_file = mouse_perturb_dir / "deg.txt"
mouse_perturb_clusterprop_file = mouse_perturb_dir / "clusterprop.txt"


In [49]:
conversion_dict: dict[str, bool] = {
    "convert_string": True,
    "convert_integer": False,
    "convert_boolean": False,
    "convert_floating": False,
}
mouse_perturb_deg = pd.read_csv(mouse_perturb_deg_file, sep="\t").convert_dtypes(**conversion_dict)
mouse_perturb_clusterprop = pd.read_csv(mouse_perturb_clusterprop_file, sep="\t").convert_dtypes(**conversion_dict)



In [50]:
mouse_perturb_deg.head()

,Gene,logFC,logCPM,LR,PValue,padj,cell type,perturbation
0,Selenoo,9.651227,5.957813,19.984261,7.810000e-06,0.034711,Inhib_Lhx6+Sst-,Foxg1_3
1,Fbxl16,3.262595,7.123919,24.824542,6.280000e-07,0.005583,Inhib_Lhx6+Sst-,Foxg1_3
2,Meis2,-1.150044,7.671438,18.072991,2.130000e-05,0.027210,Excit_L5NP_CTX,Tbr1_1
3,Gm10762,8.258858,5.390421,17.808925,2.440000e-05,0.028133,Excit_L5NP_CTX,Tbr1_1
4,Uckl1,7.372006,5.647647,16.532829,4.780000e-05,0.034794,Excit_L5NP_CTX,Tbr1_1


In [51]:
mouse_perturb_clusterprop.head()

,subcluster,guide,PropMean.treatNonTarget2,PropMean.treatGuide,PropRatio,limma_coef,Tstatistic,P.Value,FDR
0,cluster_5,Foxg1_1,0.106732,0.010755,0.100762,-2.385959,-8.394585,6.410000e-07,0.000010
1,cluster_7,Foxg1_1,0.011199,0.273223,24.396342,3.587679,7.867436,4.150000e-05,0.000245
2,cluster_21,Foxg1_1,0.033080,0.006097,0.184311,-1.491792,-5.627320,5.630000e-05,0.000245
3,cluster_2,Foxg1_1,0.120282,0.003468,0.028834,-3.387877,-7.395180,6.530000e-05,0.000245
4,cluster_0,Foxg1_1,0.159987,0.047844,0.299048,-1.237609,-5.023114,1.717350e-04,0.000515


In [52]:
# print data types of columns:
print(mouse_perturb_deg.dtypes)
print(mouse_perturb_clusterprop.dtypes)


Gene            string[python]
logFC                  float64
logCPM                 float64
LR                     float64
PValue                 float64
padj                   float64
cell type       string[python]
perturbation    string[python]
dtype: object
subcluster                  string[python]
guide                       string[python]
PropMean.treatNonTarget2           float64
PropMean.treatGuide                float64
PropRatio                          float64
limma_coef                         float64
Tstatistic                         float64
P.Value                            float64
FDR                                float64
dtype: object


In [53]:
def sql_friendly_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.lower().str.replace(r'[^a-z0-9_]', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_')
    return df

mouse_perturb_deg = sql_friendly_columns(mouse_perturb_deg)
mouse_perturb_clusterprop = sql_friendly_columns(mouse_perturb_clusterprop)

In [54]:
mouse_perturb_deg.head()

,gene,logfc,logcpm,lr,pvalue,padj,cell_type,perturbation
0,Selenoo,9.651227,5.957813,19.984261,7.810000e-06,0.034711,Inhib_Lhx6+Sst-,Foxg1_3
1,Fbxl16,3.262595,7.123919,24.824542,6.280000e-07,0.005583,Inhib_Lhx6+Sst-,Foxg1_3
2,Meis2,-1.150044,7.671438,18.072991,2.130000e-05,0.027210,Excit_L5NP_CTX,Tbr1_1
3,Gm10762,8.258858,5.390421,17.808925,2.440000e-05,0.028133,Excit_L5NP_CTX,Tbr1_1
4,Uckl1,7.372006,5.647647,16.532829,4.780000e-05,0.034794,Excit_L5NP_CTX,Tbr1_1


In [55]:
mouse_perturb_clusterprop.head()

,subcluster,guide,propmean_treatnontarget2,propmean_treatguide,propratio,limma_coef,tstatistic,p_value,fdr
0,cluster_5,Foxg1_1,0.106732,0.010755,0.100762,-2.385959,-8.394585,6.410000e-07,0.000010
1,cluster_7,Foxg1_1,0.011199,0.273223,24.396342,3.587679,7.867436,4.150000e-05,0.000245
2,cluster_21,Foxg1_1,0.033080,0.006097,0.184311,-1.491792,-5.627320,5.630000e-05,0.000245
3,cluster_2,Foxg1_1,0.120282,0.003468,0.028834,-3.387877,-7.395180,6.530000e-05,0.000245
4,cluster_0,Foxg1_1,0.159987,0.047844,0.299048,-1.237609,-5.023114,1.717350e-04,0.000515


In [58]:
from processing.new_sqlite3 import NewSqlite3
import logging
logger = logging.getLogger(__name__)

with NewSqlite3("mouse_perturb.db", logger) as new_sqlite3:
    mouse_perturb_deg.to_sql("mouse_perturb_deg", new_sqlite3.conn, if_exists="replace", index=False)
    mouse_perturb_clusterprop.to_sql("mouse_perturb_clusterprop", new_sqlite3.conn, if_exists="replace", index=False)
